In [3]:
import os, re, pickle
import pandas as pd
import numpy as np
from os.path import join
from tqdm import tqdm

deposit_dtype = {'채무자키' : str, '입금키':str, '계좌키':str, '입금계좌번호':str, '계좌번호':str, '타입금키':str, '타채무자키':str, '보증인키':str}
rehabilitation_dtype = {'채무자키':str, '계좌키':str, '분납키':str, '사건키':str}

In [4]:
# 최신 기준데이터, 분납중지는 계좌전체에 적용될 테니 채무자키를 기준으로 해도 됨
path_rehabilitation = r"D:\3.자산\전산 dataset\솔림\230904\개인회생새창_20230904_1949_기준데이터.xlsx" 
rehabilitation = pd.read_excel(path_rehabilitation, sheet_name="개인회생(전체)", dtype=rehabilitation_dtype).fillna("")

# 입금데이터
path_deposit = r"D:\3.자산\프로젝트\2023 전산데이터 클렌징 및 업무개편 프로젝트\check\개인회생\분납재진행\분납중지 중 개인회생입금_0306~0907.xlsx"
deposit = pd.read_excel(path_deposit, dtype=deposit_dtype).fillna("")

c:\Users\sl\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [7]:
# # 분납중지와 진행중인 건 충돌여부 (사건번호로 비교) - 1건 있었는데, 하나는 채권번호 단순오기로 삭제해야 할 건데, 삭제 안 한 케이스
# rehabilitation_중지 = rehabilitation[rehabilitation.분납상태=="중단"][["채무자키", "계좌키", "분납자성명인", "분납키", "관할법원", "사건번호"]].copy()
# rehabilitation_진행 = rehabilitation[rehabilitation.분납상태=="진행"][["채무자키", "계좌키", "분납자성명인", "분납키", "관할법원", "사건번호"]].copy()
# merged = pd.merge(rehabilitation_중지,rehabilitation_진행, on=['채무자키', '분납자성명인'], how='inner')
# merged.query('사건번호_x==사건번호_y')

In [22]:
rehabilitation_중지 = rehabilitation[rehabilitation.분납상태=="중단"][["채무자키", "계좌키", "분납자성명인", "분납키", "관할법원", "사건번호", "분납중단일", "중단사유"]].copy()
rehabilitation_중지.rename(columns={"분납자성명인" : "입금자"}, inplace=True)
deposit_reha = deposit.query('입금구분=="개인회생"')[["채무자키", "계좌키", "입금키", "입금합계", "입금자", "입금일"]].copy()

In [23]:
중지입금 = pd.merge(rehabilitation_중지, deposit_reha, on=["채무자키", "입금자"]) # inner
중지입금.reindex(columns={"채무자키","입금자","분납키","관할법원","사건번호","분납중단일","입금일","중단사유","입금합계","계좌키_x","계좌키_y","입금키"})

,채무자키,계좌키_x,입금자,분납키,관할법원,사건번호,분납중단일,중단사유,계좌키_y,입금키,입금합계,입금일
0,20407817,200941272,손성주,20105383,수원회생법원,2019개회35033,2023-04-10,면책,200941272,30470888,579300,2023-04-06
1,20407817,200941272,손성주,20105383,수원회생법원,2019개회35033,2023-04-10,면책,200941272,30416319,55672,2023-03-06
2,20407914,200920699,김재호,20104362,의정부지방법원,2017개회38711,2023-04-24,면책,200920699,30499653,207911,2023-04-14
3,20407971,200920828,심상훈,20104193,의정부지방법원,2017개회12528,2023-05-04,면책,200920828,30478675,22447,2023-04-07
4,20407971,200920828,심상훈,20104193,의정부지방법원,2017개회12528,2023-05-04,면책,200920828,30416594,14967,2023-03-06
...,...,...,...,...,...,...,...,...,...,...,...,...
2005,20527947,201061612,최장현,20136060,대구지방법원,2020개회180640,2023-08-24,면책,201061612,30481760,408551,2023-04-10
2006,20527947,201061612,최장현,20136060,대구지방법원,2020개회180640,2023-08-24,면책,201061612,30421422,408551,2023-03-09
2007,20527951,201061616,조재호,20137276,창원지방법원,2020개회129061,2023-07-19,인가후중단,201061616,30877637,172851,2023-08-10
2008,20527951,201061616,조재호,20137276,창원지방법원,2020개회129061,2023-07-19,인가후중단,201061616,30442649,24693,2023-03-22


In [26]:
중지입금 = 중지입금.query("분납중단일<=입금일")

In [28]:
중지입금.groupby(['채무자키', '입금자'])
중지입금.set_index(['채무자키', '입금자'], drop=True).to_excel("c:/Users/SL/Desktop/분납재진행.xlsx")